# 欠損値補完
1. 欠損値のまま取り扱う
2. 欠損値を代表値で埋める
3. 欠損値を他の変数から予測する
4. 欠損値から新たな特徴量を作成する

In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display
import xgboost as xgb
from typing import List, Union

In [2]:
# データ準備
df = pd.read_csv('sample_nan.csv')
display(df)
#       name   age state  point
# 0    Alice  24.0    NY    NaN
# 1  Charlie   NaN    CA    NaN
# 2     Dave  68.0    TX   70.0
# 3    Ellen   NaN    CA   88.0
# 4    Frank  30.0   NaN    NaN

,name,age,state,point
0,Alice,24.0,NY,NaN
1,Charlie,NaN,CA,NaN
2,Dave,68.0,TX,70.0
3,Ellen,NaN,CA,88.0
4,Frank,30.0,NaN,NaN


## 1. 欠損値のまま取り扱う

GBDTモデルは欠損値を埋めずにそのまま入力することができる

「欠損値は何かしらの理由があって欠損している」という情報を持っているため、安易に埋めるのはかえって精度を下げかねない


## 2. 欠損値を代表値で埋める
欠損値を「平均値」「中央値」「最頻値」などの代表値で埋める

### 2-1. with 全データの統計量
全てのレコードから算出した代表値で埋める

In [3]:
df_mean = df.copy()
df_mean.fillna(df_mean.mean(), inplace=True)  # 平均値
display(df_mean)

,name,age,state,point
0,Alice,24.000000,NY,79.0
1,Charlie,40.666667,CA,79.0
2,Dave,68.000000,TX,70.0
3,Ellen,40.666667,CA,88.0
4,Frank,30.000000,NaN,79.0


In [4]:
df_median = df.copy()
df_median.fillna(df_median.median(), inplace=True)  # 中央値
display(df_median)

,name,age,state,point
0,Alice,24.0,NY,79.0
1,Charlie,30.0,CA,79.0
2,Dave,68.0,TX,70.0
3,Ellen,30.0,CA,88.0
4,Frank,30.0,NaN,79.0


In [5]:
df_mode = df.copy()
df_mode.fillna(df_mode.mode().iloc[0], inplace=True)  # 最頻値
display(df_mode)

,name,age,state,point
0,Alice,24.0,NY,70.0
1,Charlie,24.0,CA,70.0
2,Dave,68.0,TX,70.0
3,Ellen,24.0,CA,88.0
4,Frank,30.0,CA,70.0


### 2-2. with カテゴリごとの統計量
説明変数のうちの、あるカテゴリ変数ごとの統計量で欠損値を補完する

今回は、`state`ごとの代表値で`age`と`point`の欠損値を補完する

In [6]:
class BayesianAverage:
    """
    Bayesian Averageを計算する
    """
    def __init__(self, m: Union[float, int], C: int):
        """
        イニシャライザ

        @param m: あらかじめの値
        @param C: 値mの観測回数
        """
        self.m = m
        self.C = C

    def predict(self, xs: List[Union[float, int]]):
        """
        Bayesian Averageを計算する関数
        あらかじめ値mのデータをC回観測した経験の上で、実測値の平均を計算する

        @param xs: 実測値のリスト
        @return: Bayesian Average
        """
        return (np.sum(xs) + (self.C * self.m)) / (len(xs) + self.C)

In [7]:
df_state = df.copy()
ba_age, ba_point = BayesianAverage(40, 3), BayesianAverage(80, 3)  # <- ハイパーパラメータ
# ba_age: 40歳の人を３人追加する設定 (本来であれば、クラスごとに傾向が異なるので、クラスごとに異なるオブジェクトを生成する必要がある)
# ba_point: 80点の人を3人追加する設定 (本来であれば、クラスごとに傾向が異なるので、クラスごとに異なるオブジェクトを生成する必要がある)
df_state = df_state.groupby('state').agg({'age': ba_age.predict, 'point': ba_point.predict})
display(df_state)
#         age  point
# state
# CA     24.0   59.6
# NY     36.0   52.5
# TX     47.0   70.0
# これらを埋める
df_bayes_ave = df.copy()
for value in ['age', 'point']:
    for state, data in df_bayes_ave.groupby('state'):
        bayes_ave = df_state.loc[state, value]
        df_bayes_ave.loc[df_bayes_ave['state'] == state, value] = data[value].fillna(value=bayes_ave)
display(df_bayes_ave)

,age,point
state,,
CA,24.0,65.6
NY,36.0,60.0
TX,47.0,77.5


,name,age,state,point
0,Alice,24.0,NY,60.0
1,Charlie,24.0,CA,65.6
2,Dave,68.0,TX,70.0
3,Ellen,24.0,CA,88.0
4,Frank,30.0,NaN,NaN


## 3. 欠損値を他の変数から予測する
今作った `df_bayes_ave` の`age`と`point`を説明変数として、`state`の最後の行にある欠損値NaNを予測する

### Step1:
~~~
stateが欠損していないレコードの集まりと、stateが欠損しているレコードの集まりで分ける
そして、前者を「訓練データ」、後者を「テストデータ」とする
~~~
### Step2: 
~~~
訓練データを予測モデルに通して、stateを予測するように学習する
~~~
### Step3:
~~~
テストデータを予測モデルに通して、NaNであるstateを予測する
~~~

In [8]:
# ====Step1: データ準備==============
# [前提] df_bayes_aveを用いて、stateの最後の行NaNを予測する
# state -> indexへの変換辞書
state_s2i = {state: i for i, state in enumerate(df_bayes_ave['state'].dropna().unique())}
print(f'stateの種類: {state_s2i}')  # stateの種類: {'NY': 0, 'CA': 1, 'TX': 2}

X_train = df_bayes_ave[['age', 'point']].dropna()  # 訓練データ(説明変数)には欠損を含めない
y_train = df_bayes_ave['state'].dropna().map(state_s2i)  # 訓練データ(目的変数)には欠損を含めない
X_test = df_bayes_ave[df_bayes_ave.isnull().any(axis=1)][['age', 'point']]  # テストデータは欠損が含まれているレコードのみ抽出
# ================================

# ======Step2: 学習=================
# xgboostのオブジェクトに変換する
dtrain = xgb.DMatrix(X_train, label=y_train)
# ハイパーパラメータ設定
param = {'max_depth': 2, 'eta': 0.1, 'objective': 'multi:softmax', 'num_class': len(state_s2i)}
# 学習
bst = xgb.train(param, dtrain, 20)
# ===============================

# ======Step3: 予測=================
# 予測
dtest = xgb.DMatrix(X_test)
pred_test = bst.predict(dtest)
print(pred_test)
# [1.]
# ================================

# index -> stateへの変換辞書
state_i2s = {i: state for state, i in state_s2i.items()}
# 予測値を欠損値に代入する
X_test['state'] = [state_i2s[int(pred)] for pred in pred_test]
# 欠損がない訓練データと欠損を埋めたテストデータを結合
train_test = pd.concat([X_train.join(y_train.map(state_i2s)), X_test])
df_bayes_ave2 = pd.concat([df_bayes_ave['name'], train_test], axis=1)[['name', 'age', 'state', 'point']]
display(df_bayes_ave2)

stateの種類: {'NY': 0, 'CA': 1, 'TX': 2}
[1.]


,name,age,state,point
0,Alice,24.0,NY,60.0
1,Charlie,24.0,CA,65.6
2,Dave,68.0,TX,70.0
3,Ellen,24.0,CA,88.0
4,Frank,30.0,CA,NaN


`state`の最後の行にある欠損値は「CA」と予測できた

## 4. 欠損値から新たな特徴量を作成する

### 4-1. 欠損しているかどうかを表す二値変数を新たに作る
のちに欠損値を埋めたとしても、「欠損していた」という情報が失われない


In [9]:
df_isnull = df.copy()
df_isnull['point_isnull'] = df_isnull.isnull()['point'].map({True: 1, False: 0})
df_isnull['point'].fillna(np.mean(df_isnull['point']), inplace=True)  # 平均値で埋める
display(df_isnull)

,name,age,state,point,point_isnull
0,Alice,24.0,NY,79.0,1
1,Charlie,NaN,CA,79.0,1
2,Dave,68.0,TX,70.0,0
3,Ellen,NaN,CA,88.0,0
4,Frank,30.0,NaN,79.0,1


### 4-2. レコードごとに欠損している変数の数を数える

In [10]:
df_null_num = df.copy()
df_null_num['null_num'] = df_null_num.isnull().sum(axis=1)
display(df_null_num)

,name,age,state,point,null_num
0,Alice,24.0,NY,NaN,1
1,Charlie,NaN,CA,NaN,2
2,Dave,68.0,TX,70.0,0
3,Ellen,NaN,CA,88.0,1
4,Frank,30.0,NaN,NaN,2
